## Task 1 - Bigram Model

Select training data to build a bigram language model that can help you in text sequence generation for sequence data. You should implement a python class with multiple methods to do the following jobs:

1. The class name is __LanguageModel__.
2. A __constructor__ to get the text file name and open that file.
3. A method called __Clean__ to process the text (Tokenization, Lemmatization)! __Don’t remove stop words__.
4. A method called __LMBigram__ to build 2-D Language Model Matrix with Laplace smoothing using NLTK. This method saves generated matrix in an instance variable.
5. A method called __Run__ that takes a part of sentence from user and returns the expected next word using the constructed matrix from the previous step.

In [140]:
import pandas as pd
import glob
import emoji  # for removing emojis
import re
import qalsadi.lemmatizer
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.arlstem import ARLSTem
from nltk.util import ngrams
from tqdm import tqdm  # progress bar

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tamer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 1- Code testing before Class implementation

### 1.1- Collecting the data into one dataframe

In [2]:
text_files = glob.glob("data/file*.txt")

In [3]:
text_files

['data\\file1.txt',
 'data\\file10.txt',
 'data\\file2.txt',
 'data\\file3.txt',
 'data\\file4.txt',
 'data\\file5.txt',
 'data\\file6.txt',
 'data\\file7.txt',
 'data\\file8.txt',
 'data\\file9.txt']

In [4]:
text_file_df_list = []

for text_file in text_files:
    df = pd.read_csv(text_file, sep="\t")
    text_file_df_list.append(df)
    
full_df = pd.concat(text_file_df_list, ignore_index=True)    

In [5]:
full_df

,tweetID,tweetText
0,549679192804061184,"الاعدام لعامل مطعم قتل زميله طعناً في ""البيادر..."
1,549699343666532352,#الأخبار ▪ تأجيل محاكمة 7 إرهابيين بسبب غياب ا...
2,549711593487888387,@helale9999 عشآن أعطيتك وحده صميم صرت ترمي أعذ...
3,549719610459967488,#النهدي ثمانية قتلى في تفجير انتحاري بسيارة مف...
4,549720880717508608,البحرين: ضبط مطلوبين متورطين في التفجير بالعكر...
5,549757675227979778,@El__DoN__ABoOoD @soliman_sport توهير تفكيره ي...
6,549780004230283264,"صحيفة ""نيويورك تايمز"" الطفل الانتحاري الذي سلم..."
7,549791374027927552,ليتوانيا تستقبل عام 2015 باعتماد اليورو http:/...
8,549795718206980099,يحتاج الداعي أكثر من غيره إلى حسن الظاهر والوق...
9,549807839934676992,المؤبد بحق موظف قتل رب عمله نحراً - عاقبت الهي...


In [6]:
full_df.to_csv("data/full_data.txt", index=None, sep="\t")

In [7]:
# testing reading the created file containing all tweets
full_df = pd.read_csv("data/full_data.txt", sep="\t")

In [8]:
full_df

,tweetID,tweetText
0,549679192804061184,"الاعدام لعامل مطعم قتل زميله طعناً في ""البيادر..."
1,549699343666532352,#الأخبار ▪ تأجيل محاكمة 7 إرهابيين بسبب غياب ا...
2,549711593487888387,@helale9999 عشآن أعطيتك وحده صميم صرت ترمي أعذ...
3,549719610459967488,#النهدي ثمانية قتلى في تفجير انتحاري بسيارة مف...
4,549720880717508608,البحرين: ضبط مطلوبين متورطين في التفجير بالعكر...
5,549757675227979778,@El__DoN__ABoOoD @soliman_sport توهير تفكيره ي...
6,549780004230283264,"صحيفة ""نيويورك تايمز"" الطفل الانتحاري الذي سلم..."
7,549791374027927552,ليتوانيا تستقبل عام 2015 باعتماد اليورو http:/...
8,549795718206980099,يحتاج الداعي أكثر من غيره إلى حسن الظاهر والوق...
9,549807839934676992,المؤبد بحق موظف قتل رب عمله نحراً - عاقبت الهي...


### 1.2- Extracting text from full_df

In [9]:
full_df.drop("tweetID", axis=1, inplace=True)

In [10]:
full_df

,tweetText
0,"الاعدام لعامل مطعم قتل زميله طعناً في ""البيادر..."
1,#الأخبار ▪ تأجيل محاكمة 7 إرهابيين بسبب غياب ا...
2,@helale9999 عشآن أعطيتك وحده صميم صرت ترمي أعذ...
3,#النهدي ثمانية قتلى في تفجير انتحاري بسيارة مف...
4,البحرين: ضبط مطلوبين متورطين في التفجير بالعكر...
5,@El__DoN__ABoOoD @soliman_sport توهير تفكيره ي...
6,"صحيفة ""نيويورك تايمز"" الطفل الانتحاري الذي سلم..."
7,ليتوانيا تستقبل عام 2015 باعتماد اليورو http:/...
8,يحتاج الداعي أكثر من غيره إلى حسن الظاهر والوق...
9,المؤبد بحق موظف قتل رب عمله نحراً - عاقبت الهي...


In [11]:
test_sent = full_df.iloc[0][0]

In [12]:
test_sent

'الاعدام لعامل مطعم قتل زميله طعناً في "البيادر" أيدت محكمة التمييز الحكم الصادر عن محكمة الجنايات الكبرى والقاضي... http://t.co/H0txdjv3Kn'

### 1.3- Text Pre-processing

Assumptions:
* __URLs__ would be removed
* __Hashtags__ would be removed
* __Mentions__ would be removed
* __Repeating dots__ (more than two occurences) e.g.: __...__ would be removed
* __Emojis__ would be removed
* __Repeating characters__ (more than two occurences) e.g.: عاااااااجل would be removed

##### 1.3.1 Testing regular expression substitution

In [13]:
# Removing URLs, Hashtags, Mentions, and Repeating dots
#  removing URLs from text StackOverflow reference: https://stackoverflow.com/a/11332580
#  removing repeating characters than occur more than two times StackOverflow reference: https://stackoverflow.com/a/4574516
#   URL: https?:\/\/.*[\r\n]*
#   Hashtag: #\w+
#   Mention: @\w+
#   Repeating dots: \.{2,}
#   Repeating character: (\w)\1{2,}

# Note: the pattern needs to be in raw string format
re_general_pattern = r"https?:\/\/.*[\r\n]*|#\w+|@\w+|\.{2,}"
re_repeating_character_pattern = r"(\w)\1{2,}"

In [14]:
# Removing emojis
#  StackOverflow reference: https://stackoverflow.com/a/67396231
def remove_emojis(string):
    return emoji.get_emoji_regexp().sub(u"", string)

In [15]:
emoji.get_emoji_regexp()

C:\Users\tamer\Anaconda3\envs\nlp-course\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  """Entry point for launching an IPython kernel.


re.compile(r'(👨🏿\u200d❤️\u200d💋\u200d👨🏿|👨🏿\u200d❤️\u200d💋\u200d👨🏻|👨🏿\u200d❤️\u200d💋\u200d👨🏾|👨🏿\u200d❤️\u200d💋\u200d👨🏼|👨🏿\u200d❤️\u200d💋\u200d👨🏽|👨🏻\u200d❤️\u200d💋\u200d👨🏻|👨🏻\u200d❤️\u200d💋\u200d👨🏿|👨🏻\u200d❤️\u200d💋\u200d👨🏾|👨🏻\u200d❤️\u200d💋\u200d👨🏼|👨🏻\u200d❤️\u200d💋\u200d👨🏽|👨🏾\u200d❤️\u200d💋\u200d👨🏾|👨🏾\u200d❤️\u200d💋\u200d👨🏿|👨🏾\u200d❤️\u200d💋\u200d👨🏻|👨🏾\u200d❤️\u200d💋\u200d👨🏼|👨🏾\u200d❤️\u200d💋\u200d👨🏽|👨🏼\u200d❤️\u200d💋\u200d👨🏼|👨🏼\u200d❤️\u200d💋\u200d👨🏿|👨🏼\u200d❤️\u200d💋\u200d👨🏻|👨🏼\u200d❤️\u200d💋\u200d👨🏾|👨🏼\u200d❤️\u200d💋\u200d👨🏽|👨🏽\u200d❤️\u200d💋\u200d👨🏽|👨🏽\u200d❤️\u200d💋\u200d👨🏿|👨🏽\u200d❤️\u200d💋\u200d👨🏻|👨🏽\u200d❤️\u200d💋\u200d👨🏾|👨🏽\u200d❤️\u200d💋\u200d👨🏼|🧑🏿\u200d❤️\u200d💋\u200d🧑🏻|🧑🏿\u200d❤️\u200d💋\u200d🧑🏾|🧑🏿\u200d❤️\u200d💋\u200d🧑🏼|🧑🏿\u200d❤️\u200d💋\u200d🧑🏽|🧑🏻\u200d❤️\u200d💋\u200d🧑🏿|🧑🏻\u200d❤️\u200d💋\u200d🧑🏾|🧑🏻\u200d❤️\u200d💋\u200d🧑🏼|🧑🏻\u200d❤️\u200d💋\u200d🧑🏽|🧑🏾\u200d❤️\u200d💋\u200d🧑🏿|🧑🏾\u200d❤️\u200d💋\u200d🧑🏻|🧑🏾\u200d❤️\u200d💋\u200d🧑🏼|🧑🏾\u200d❤️\u200d💋\u200d🧑🏽|🧑🏼\u200d❤️\u200d💋\u200d🧑

###### 1.3.1.1 Testing removing repeating characters

In [16]:
re.sub(re_repeating_character_pattern, r"\1", "عااااااجل خطييير ناااااار moooooo nice yumyyyy nee")

'عاجل خطير نار mo nice yumy nee'

###### 1.3.1.2 Testing removing Emojis

In [17]:
full_df.iloc[49974][0]

': #عاجل ⭕️ #الملك_سلمان يصدر قرارات ملكية يتم الان اعلانها على القناة #السعودية"'

In [18]:
test_sent = remove_emojis(full_df.iloc[49974][0])
test_sent

C:\Users\tamer\Anaconda3\envs\nlp-course\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  after removing the cwd from sys.path.


': #عاجل ️ #الملك_سلمان يصدر قرارات ملكية يتم الان اعلانها على القناة #السعودية"'

In [19]:
test_sent = re.sub(re_general_pattern, "", test_sent)
test_sent = re.sub(re_repeating_character_pattern, r"\1", test_sent)
test_sent

':  ️  يصدر قرارات ملكية يتم الان اعلانها على القناة "'

###### 1.3.1.3 Testing removing Mentions

In [20]:
# Removing mentions from text
full_df.iloc[49994][0]

'“@AlArabiya_Brk: السعودية: الملك سلمان يصدر قرارات ملكية تعلن بعد قليل”'

In [21]:
test_sent2 = remove_emojis(full_df.iloc[49994][0])
test_sent2 = re.sub(re_general_pattern, "", test_sent2)
test_sent2 = re.sub(re_repeating_character_pattern, r"\1", test_sent2)

C:\Users\tamer\Anaconda3\envs\nlp-course\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  after removing the cwd from sys.path.


In [22]:
test_sent2

'“: السعودية: الملك سلمان يصدر قرارات ملكية تعلن بعد قليل”'

###### 1.3.1.4 Testing removing URLs

In [23]:
full_df.iloc[0][0]

'الاعدام لعامل مطعم قتل زميله طعناً في "البيادر" أيدت محكمة التمييز الحكم الصادر عن محكمة الجنايات الكبرى والقاضي... http://t.co/H0txdjv3Kn'

In [24]:
test_sent3 = remove_emojis(full_df.iloc[0][0])
test_sent3 = re.sub(re_general_pattern, "", test_sent3)
test_sent3 = re.sub(re_repeating_character_pattern, r"\1", test_sent3)

C:\Users\tamer\Anaconda3\envs\nlp-course\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  after removing the cwd from sys.path.


In [25]:
test_sent3

'الاعدام لعامل مطعم قتل زميله طعناً في "البيادر" أيدت محكمة التمييز الحكم الصادر عن محكمة الجنايات الكبرى والقاضي '

###### 1.3.1.5 Testing removing Hashtags

In [26]:
# Removing Hashtags from text
full_df.iloc[49971][0]

'#Qatar الميليشيات تبايع نوري المالكي!: الميليشيات تبايع نوري المالكي!التاريخ: 2015-01-29 في خط... http://t.co/5ibH7IezDp #Oman'

In [27]:
test_sent4 = remove_emojis(full_df.iloc[49971][0])
test_sent4 = re.sub(re_general_pattern, "", test_sent4)
test_sent4 = re.sub(re_repeating_character_pattern, r"\1", test_sent4)
test_sent4

C:\Users\tamer\Anaconda3\envs\nlp-course\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  after removing the cwd from sys.path.


' الميليشيات تبايع نوري المالكي!: الميليشيات تبايع نوري المالكي!التاريخ: 2015-01-29 في خط '

### 1.4 Testing Word Tokenization

Assumptions:
* __Punctuations__ would be removed
    * Punctuation removal StackOverflow Reference: https://stackoverflow.com/a/41024113

In [28]:
test_sent

':  ️  يصدر قرارات ملكية يتم الان اعلانها على القناة "'

In [29]:
word_tokenize(test_sent)

[':',
 '️',
 'يصدر',
 'قرارات',
 'ملكية',
 'يتم',
 'الان',
 'اعلانها',
 'على',
 'القناة',
 '``']

In [30]:
# Punctuation removal
[word.lower() for word in word_tokenize(test_sent) if word.isalpha()]

['يصدر', 'قرارات', 'ملكية', 'يتم', 'الان', 'اعلانها', 'على', 'القناة']

#### 1.4.1 Testing punctuation removal

In [31]:
test_sent

':  ️  يصدر قرارات ملكية يتم الان اعلانها على القناة "'

In [32]:
test_sent = [word.lower() for word in word_tokenize(test_sent) if word.isalpha()]
test_sent = " ".join(test_sent)
test_sent

'يصدر قرارات ملكية يتم الان اعلانها على القناة'

In [33]:
test_sent2

'“: السعودية: الملك سلمان يصدر قرارات ملكية تعلن بعد قليل”'

In [34]:
test_sent2 = [word.lower() for word in word_tokenize(test_sent2) if word.isalpha()]
test_sent2 = " ".join(test_sent2)
test_sent2

'السعودية الملك سلمان يصدر قرارات ملكية تعلن بعد قليل'

In [35]:
test_sent3

'الاعدام لعامل مطعم قتل زميله طعناً في "البيادر" أيدت محكمة التمييز الحكم الصادر عن محكمة الجنايات الكبرى والقاضي '

In [36]:
test_sent3 = [word.lower() for word in word_tokenize(test_sent3) if word.isalpha()]
test_sent3 = " ".join(test_sent3)
test_sent3

'الاعدام لعامل مطعم قتل زميله في البيادر أيدت محكمة التمييز الحكم الصادر عن محكمة الجنايات الكبرى والقاضي'

In [37]:
test_sent4

' الميليشيات تبايع نوري المالكي!: الميليشيات تبايع نوري المالكي!التاريخ: 2015-01-29 في خط '

In [38]:
test_sent4 = [word.lower() for word in word_tokenize(test_sent4) if word.isalpha()]
test_sent4 = " ".join(test_sent4)
test_sent4

'الميليشيات تبايع نوري المالكي الميليشيات تبايع نوري المالكي التاريخ في خط'

### 1.5 Testing Stemming/Lemmatizing Arabic Language

#### 1.5.1 Testing ARLSTem for stemming Arabic Language

In [39]:
ARstemmer = ARLSTem()

In [40]:
test_sent

'يصدر قرارات ملكية يتم الان اعلانها على القناة'

In [41]:
singles = [ARstemmer.stem(token) for token in word_tokenize(test_sent)]
print(" ".join(singles))

صدر قرار ملكي يتم الن اعل علي قنا


In [42]:
test_sent2

'السعودية الملك سلمان يصدر قرارات ملكية تعلن بعد قليل'

In [43]:
singles = [ARstemmer.stem(token) for token in word_tokenize(test_sent2)]
print(" ".join(singles))

سعودي ملك سلم صدر قرار ملكي تعل بعد قليل


In [44]:
test_sent3

'الاعدام لعامل مطعم قتل زميله في البيادر أيدت محكمة التمييز الحكم الصادر عن محكمة الجنايات الكبرى والقاضي'

In [45]:
singles = [ARstemmer.stem(token) for token in word_tokenize(test_sent3)]
print(" ".join(singles))

عدم لعامل مطعم قتل زميل في بيادر ايد محكم تمييز حكم صادر عن محكم جناي كبري قاضي


In [46]:
test_sent4

'الميليشيات تبايع نوري المالكي الميليشيات تبايع نوري المالكي التاريخ في خط'

In [47]:
singles = [ARstemmer.stem(token) for token in word_tokenize(test_sent4)]
print(" ".join(singles))

ميليشي بايع وري مال ميليشي بايع وري مال تاريخ في خط


#### 1.5.2 Testing Qalsadi Lemmatizer for lemmatizing Arabic Language

In [48]:
qalsadi_lemmatizer = qalsadi.lemmatizer.Lemmatizer()

In [49]:
test_sent

'يصدر قرارات ملكية يتم الان اعلانها على القناة'

In [50]:
singles = [qalsadi_lemmatizer.lemmatize(token) for token in word_tokenize(test_sent)]
print(" ".join(singles))

صدر قرار ملك يتم ال اعلانها على قناة


In [51]:
test_sent2

'السعودية الملك سلمان يصدر قرارات ملكية تعلن بعد قليل'

In [52]:
singles = [qalsadi_lemmatizer.lemmatize(token) for token in word_tokenize(test_sent2)]
print(" ".join(singles))

سعودي ملك سلم صدر قرار ملك علن بعد قليل


In [53]:
test_sent3

'الاعدام لعامل مطعم قتل زميله في البيادر أيدت محكمة التمييز الحكم الصادر عن محكمة الجنايات الكبرى والقاضي'

In [54]:
singles = [qalsadi_lemmatizer.lemmatize(token) for token in word_tokenize(test_sent3)]
print(" ".join(singles))

الاعدام عامل مطعم قتل زميل في بيادر أيد محكم تمييز حكم صادر عن محكم جناية كبرى قاضي


In [55]:
test_sent4

'الميليشيات تبايع نوري المالكي الميليشيات تبايع نوري المالكي التاريخ في خط'

In [56]:
singles = [qalsadi_lemmatizer.lemmatize(token) for token in word_tokenize(test_sent4)]
print(" ".join(singles))

الميليشيات بايع نور مالك الميليشيات بايع نور مالك تاريخ في خط


__The Qalsadi Lemmatizer will be chosen for the Bigram Language Model__

In [57]:
test_sent = [qalsadi_lemmatizer.lemmatize(token) for token in word_tokenize(test_sent)]
print(" ".join(test_sent))

صدر قرار ملك يتم ال اعلانها على قناة


In [58]:
test_sent2 = [qalsadi_lemmatizer.lemmatize(token) for token in word_tokenize(test_sent2)]
print(" ".join(test_sent2))

سعودي ملك سلم صدر قرار ملك علن بعد قليل


In [59]:
test_sent3 = [qalsadi_lemmatizer.lemmatize(token) for token in word_tokenize(test_sent3)]
print(" ".join(test_sent3))

الاعدام عامل مطعم قتل زميل في بيادر أيد محكم تمييز حكم صادر عن محكم جناية كبرى قاضي


In [60]:
test_sent4 = [qalsadi_lemmatizer.lemmatize(token) for token in word_tokenize(test_sent4)]
print(" ".join(test_sent4))

الميليشيات بايع نور مالك الميليشيات بايع نور مالك تاريخ في خط


### 1.6 Testing bigram model creation

#### 1.6.1 Testing nltk bigram function

In [168]:
test_sent

['صدر', 'قرار', 'ملك', 'يتم', 'ال', 'اعلانها', 'على', 'قناة']

In [169]:
list(ngrams(test_sent, 2))

[('صدر', 'قرار'),
 ('قرار', 'ملك'),
 ('ملك', 'يتم'),
 ('يتم', 'ال'),
 ('ال', 'اعلانها'),
 ('اعلانها', 'على'),
 ('على', 'قناة')]

In [170]:
test_list = []

In [171]:
test_list.extend(list(ngrams(test_sent, 2)))

In [172]:
test_list

[('صدر', 'قرار'),
 ('قرار', 'ملك'),
 ('ملك', 'يتم'),
 ('يتم', 'ال'),
 ('ال', 'اعلانها'),
 ('اعلانها', 'على'),
 ('على', 'قناة')]

In [173]:
test_list.extend(list(ngrams(test_sent2, 2)))

In [174]:
test_list

[('صدر', 'قرار'),
 ('قرار', 'ملك'),
 ('ملك', 'يتم'),
 ('يتم', 'ال'),
 ('ال', 'اعلانها'),
 ('اعلانها', 'على'),
 ('على', 'قناة'),
 ('سعودي', 'ملك'),
 ('ملك', 'سلم'),
 ('سلم', 'صدر'),
 ('صدر', 'قرار'),
 ('قرار', 'ملك'),
 ('ملك', 'علن'),
 ('علن', 'بعد'),
 ('بعد', 'قليل')]

#### 1.6.2 Testing the text processing function clean() - Steps 1.1 to 1.5

In [175]:
# Creating text preprocessing function (clean()) from steps 1.1 to 1.5
def clean(sentence):
    def remove_emojis(string):
        return emoji.get_emoji_regexp().sub(u"", string)

    re_general_pattern = r"https?:\/\/.*[\r\n]*|#\w+|@\w+|\.{2,}"
    re_repeating_character_pattern = r"(\w)\1{2,}"
    lemmatizer = qalsadi.lemmatizer.Lemmatizer()
    
    # 1- Removing URLs, Hashtags, Mentions, and repeating dots
    sentence = re.sub(re_general_pattern, "", sentence)
    # 2- Removing repeating characters that occur more than twice
    sentence = re.sub(re_repeating_character_pattern, r"\1", sentence)
    # 3- Removing emojis
    sentence = remove_emojis(sentence)
    # 4- Tokenization and punctuation removal (only alphanumeric)
    sequence = [token.lower() for token in word_tokenize(sentence) if token.isalpha()]
    # 5- Lemmatization
    sequence = [qalsadi_lemmatizer.lemmatize(token) for token in sequence]
    
    return sequence

In [176]:
full_df.iloc[0][0]

'الاعدام لعامل مطعم قتل زميله طعناً في "البيادر" أيدت محكمة التمييز الحكم الصادر عن محكمة الجنايات الكبرى والقاضي... http://t.co/H0txdjv3Kn'

In [177]:
clean(full_df.iloc[0][0])

C:\Users\tamer\Anaconda3\envs\nlp-course\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  after removing the cwd from sys.path.


['الاعدام',
 'عامل',
 'مطعم',
 'قتل',
 'زميل',
 'في',
 'بيادر',
 'أيد',
 'محكم',
 'تمييز',
 'حكم',
 'صادر',
 'عن',
 'محكم',
 'جناية',
 'كبرى',
 'قاضي']

#### 1.6.3 Testing bigram model creation from tweet data

In [178]:
test_df = pd.read_csv("data/file1.txt", sep="\t").drop("tweetID", axis=1)

In [179]:
test_df

,tweetText
0,"الاعدام لعامل مطعم قتل زميله طعناً في ""البيادر..."
1,#الأخبار ▪ تأجيل محاكمة 7 إرهابيين بسبب غياب ا...
2,@helale9999 عشآن أعطيتك وحده صميم صرت ترمي أعذ...
3,#النهدي ثمانية قتلى في تفجير انتحاري بسيارة مف...
4,البحرين: ضبط مطلوبين متورطين في التفجير بالعكر...
5,@El__DoN__ABoOoD @soliman_sport توهير تفكيره ي...
6,"صحيفة ""نيويورك تايمز"" الطفل الانتحاري الذي سلم..."
7,ليتوانيا تستقبل عام 2015 باعتماد اليورو http:/...
8,يحتاج الداعي أكثر من غيره إلى حسن الظاهر والوق...
9,المؤبد بحق موظف قتل رب عمله نحراً - عاقبت الهي...


In [180]:
for index, tweet in test_df.iterrows():
    if index == 10:
        break
    print(tweet[0])

الاعدام لعامل مطعم قتل زميله طعناً في "البيادر" أيدت محكمة التمييز الحكم الصادر عن محكمة الجنايات الكبرى والقاضي... http://t.co/H0txdjv3Kn
#الأخبار ▪ تأجيل محاكمة 7 إرهابيين بسبب غياب الدفاع: أجلت محكمة الجنايات بالعاصمة إلى تاريخ لاحق محاكمة سبعة إ... http://t.co/GM4jmpAWbR
@helale9999 عشآن أعطيتك وحده صميم صرت ترمي أعذار ...حقق العالميةة و أرجع كلمني يَ الأياب الانتحاري
#النهدي ثمانية قتلى في تفجير انتحاري بسيارة مفخخة أمام معملين للغاز في ريف حمص - شبكة الصين http://t.co/r5zFEuzAPu
البحرين: ضبط مطلوبين متورطين في التفجير بالعكر الشرقي بقية الموضوع اضغط هنا http://t.co/t4A5bNrqyh
@El__DoN__ABoOoD @soliman_sport توهير تفكيره يورو على يورو يطلع ٢ يورو وعلى كذا ثقافة آسيوية متأصلة وبصراحة أشوف إنها جيدة له لوحده فقط
صحيفة "نيويورك تايمز" الطفل الانتحاري الذي سلم نفسه لحظات قبل التفجير قرب حسينيه في البياع هو سوري وليس عراقي... http://t.co/PH1a97rvtb
ليتوانيا تستقبل عام 2015 باعتماد اليورو http://t.co/29IJLgr73b
يحتاج الداعي أكثر من غيره إلى حسن الظاهر والوقار والهدوء ليتأسى به ، وخير ال

In [182]:
bigrams = []
unigrams = []  # for laplacian smoothing

for index, tweet in tqdm(test_df.iterrows()):
    sequence = clean(tweet[0]) 
    
    # bigram model creation
    bigrams.extend(list(ngrams(sequence, 2)))
    
    # unigram model creation for the purposes of laplacian smoothing
    unigrams.extend(list(ngrams(sequence, 1)))

0it [00:00, ?it/s]C:\Users\tamer\Anaconda3\envs\nlp-course\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  after removing the cwd from sys.path.
5000it [06:19, 13.17it/s]


In [183]:
freq_bi = nltk.FreqDist(bigrams)
freq_uni = nltk.FreqDist(unigrams)

print ("Most common bigrams without stopword removal and without add-1 smoothing: ", freq_bi.most_common(10))
print ("Most common unigrams without stopword removal and without add-1 smoothing: ", freq_uni.most_common(10))

Most common bigrams without stopword removal and without add-1 smoothing:  [(('أنس', 'ليبي'), 420), (('تفجير', 'انتحار'), 418), (('أبو', 'أنس'), 398), (('في', 'تفجير'), 237), (('من', 'محاكم'), 236), (('في', 'نيويورك'), 232), (('كور', 'شمال'), 231), (('على', 'كور'), 225), (('قياد', 'في'), 209), (('جنائي', 'دولي'), 204)]
Most common unigrams without stopword removal and without add-1 smoothing:  [(('في',), 2722), (('على',), 1345), (('من',), 1047), (('تفجير',), 886), (('سوى',), 684), (('ليبي',), 598), (('انتحار',), 579), (('أنس',), 536), (('أبو',), 501), (('كنيس',), 460)]


#### 1.6.4 Testing bigram model with laplacian smoothing (add-1 smoothing)

Note: I am assuming the vocabulary size would be the number of unique n-1 grams according to this quora thread [link](https://www.quora.com/What-is-the-meaning-of-Vocabulary-in-n-gram-Laplace-Smoothing)

In [184]:
# Creating unigram vocabulary set for laplacian smoothing
unigrams_voc = set([])

for unigram in unigrams:
    if unigram not in unigrams_voc:
        unigrams_voc.add(unigram)

In [185]:
vocab_size = len(unigrams_voc)
vocab_size

5898

In [186]:
len(bigrams)

52330

In [191]:
test_bigram = ('أنس', 'ليبي')

In [192]:
bigrams.count(test_bigram)

420

In [205]:
test_bigram[0]

'أنس'

In [215]:
# Needs to be as a tuple instead of a string
(test_bigram[0],)

('أنس',)

In [214]:
unigrams.count((test_bigram[0],))

536

In [245]:
bigrams_smoothed_prob_model = {}

for bigram in tqdm(bigrams):
    numerator = bigrams.count(bigram) + 1  # Count of (wi-1, wi)
    denominator = unigrams.count((bigram[0],)) + vocab_size  # Count of wi - 1
    smoothed_prob = numerator / denominator
    
    bigrams_smoothed_prob_model[bigram] = smoothed_prob

100%|████████████████████████████████████████████████████████████████████████████| 52330/52330 [11:58<00:00, 72.85it/s]


In [251]:
bigrams_smoothed_prob_model = dict(sorted(bigrams_smoothed_prob_model.items(), key=lambda item: item[1], reverse=True))

In [252]:
list(bigrams_smoothed_prob_model.items())[:10]

[(('أنس', 'ليبي'), 0.0654336338203295),
 (('أبو', 'أنس'), 0.06235349273323957),
 (('تفجير', 'انتحار'), 0.061762971698113206),
 (('كور', 'شمال'), 0.037619588130371334),
 (('من', 'محاكم'), 0.03412526997840173),
 (('قياد', 'في'), 0.03397508493771234),
 (('جنائي', 'دولي'), 0.033507682249101015),
 (('أيام', 'من'), 0.03223694976272296),
 (('على', 'كور'), 0.03120254038381886),
 (('قبل', 'أيام'), 0.03102247546691991)]

In [254]:
bigrams_smoothed_prob_model[('أنس', 'ليبي')]

0.0654336338203295

#### 1.6.5 Testing next word prediction

Assumption: According to my understanding of the implementation of the code, for bigram prediction for the next word; the sorted list of bigrams based on probability would be iterated through from highest to lowest:
* The last bigram of the input sentence would be used for checking with the sorted list of bigrams, the second word of the last bigram would be checked if it was the first word occuring in the list of sorted bigrams to "predict" the most possible next n words

In [256]:
test_bigrams_model = list(bigrams_smoothed_prob_model.items())

In [259]:
test_bigrams_model

[(('أنس', 'ليبي'), 0.0654336338203295),
 (('أبو', 'أنس'), 0.06235349273323957),
 (('تفجير', 'انتحار'), 0.061762971698113206),
 (('كور', 'شمال'), 0.037619588130371334),
 (('من', 'محاكم'), 0.03412526997840173),
 (('قياد', 'في'), 0.03397508493771234),
 (('جنائي', 'دولي'), 0.033507682249101015),
 (('أيام', 'من'), 0.03223694976272296),
 (('على', 'كور'), 0.03120254038381886),
 (('قبل', 'أيام'), 0.03102247546691991),
 (('ليبي', 'قبل'), 0.03063423645320197),
 (('محكم', 'جنائي'), 0.029864972200158855),
 (('محكم', 'جناية'), 0.027958697378872122),
 (('وفاة', 'أبو'), 0.027900378310214375),
 (('في', 'تفجير'), 0.027610208816705336),
 (('وفاة', 'قياد'), 0.027585119798234553),
 (('مر', 'أولى'), 0.027455466579506455),
 (('في', 'نيويورك'), 0.02703016241299304),
 (('لبن', 'فرض'), 0.026538586779550635),
 (('على', 'سوري'), 0.026094159878503383),
 (('مركز', 'ثقاف'), 0.024723916268336903),
 (('منطق', 'يورو'), 0.02445027896291434),
 (('بوكو', 'حرام'), 0.024156187954996692),
 (('دخول', 'على'), 0.02292310193464

In [316]:
test_string = "الاعدام لعامل مطعم قتل"
clean(test_string)

C:\Users\tamer\Anaconda3\envs\nlp-course\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  after removing the cwd from sys.path.


['الاعدام', 'عامل', 'مطعم', 'قتل']

In [317]:
test_string_bigrams = list(ngrams(clean(test_string), 2))
test_string_bigrams

C:\Users\tamer\Anaconda3\envs\nlp-course\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  after removing the cwd from sys.path.


[('الاعدام', 'عامل'), ('عامل', 'مطعم'), ('مطعم', 'قتل')]

In [318]:
test_string_bigrams[-1]

('مطعم', 'قتل')

In [319]:
test_string_bigrams[-1][0]

'مطعم'

In [320]:
test_string_bigrams[-1][1]

'قتل'

In [321]:
test_bigrams_model[0][0]

('أنس', 'ليبي')

In [322]:
test_bigrams_model[0][0][0]

'أنس'

In [323]:
test_bigrams_model[0][0][1]

'ليبي'

In [324]:
pred = []

count = 0
for bigram in test_bigrams_model:
    if bigram[0][0] == test_string_bigrams[-1][1]:      
        # to find predictions based on highest probability of n-grams     
        count += 1
        pred.append(bigram[0][1])  # Append second word in the bigram as the predicted word
        if count == 5:
            break
            
if count < 5:
    while(count != 5):
        pred.append("\0")           
        # if no word prediction is found, replace with \0
        count += 1

In [325]:
pred

['عشر', 'على', 'ملك', 'شخص', 'رب']

## 2- Class code

In [ ]:
class LanguageModel:
    def __init__(self, textfile):
        pass
    
    def clean(self):
        pass
    
    def lm_bigram(self):
        pass
    
    def run(self):
        pass

## 3-Testing class code